# Update Routes by Region

### Step 0: Libraries and Loading csv

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/raw/routes_by_region_2024.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Region                    60 non-null     object
 1   Origin Airport Code       60 non-null     object
 2   Origin Airport Name       60 non-null     object
 3   Destination Airport Code  60 non-null     object
 4   Destination Airport Name  60 non-null     object
 5   Seats                     60 non-null     int64 
dtypes: int64(1), object(5)
memory usage: 2.9+ KB


### Step 1: Drop airport destinations with good weather

In [3]:
omitted_df = df[~df['Destination Airport Code'].isin(['RUH','KWI','DXB'])].reset_index(drop=True)
omitted_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Region                    50 non-null     object
 1   Origin Airport Code       50 non-null     object
 2   Origin Airport Name       50 non-null     object
 3   Destination Airport Code  50 non-null     object
 4   Destination Airport Name  50 non-null     object
 5   Seats                     50 non-null     int64 
dtypes: int64(1), object(5)
memory usage: 2.5+ KB


### Step 2: Drop duplicate routes

In [4]:
# Drop duplicate rows based on the combination of 'Origin Airport Code' and 'Destination Airport Code'
unique_routes_df = omitted_df.drop_duplicates(subset=['Origin Airport Code', 'Destination Airport Code']).reset_index(drop=True)
unique_routes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Region                    41 non-null     object
 1   Origin Airport Code       41 non-null     object
 2   Origin Airport Name       41 non-null     object
 3   Destination Airport Code  41 non-null     object
 4   Destination Airport Name  41 non-null     object
 5   Seats                     41 non-null     int64 
dtypes: int64(1), object(5)
memory usage: 2.0+ KB


### Step 3: Handling overrepresented destination airports

In [6]:
def count_airports(df):
    return df.groupby(['Destination Airport Code','Region']).Seats.count().sort_values(ascending=False).head(10)

In [7]:
count_airports(unique_routes_df)

Destination Airport Code  Region       
LHR                       Europe           6
SIN                       Asia Pacific     3
MAD                       Europe           2
TPE                       Asia Pacific     2
SJU                       Latin America    2
SCL                       Latin America    2
MAD                       Latin America    1
YUL                       North America    1
TPE                       North America    1
RUN                       Africa           1
Name: Seats, dtype: int64

In [8]:
lhr_to_omit = unique_routes_df[unique_routes_df['Destination Airport Code'] == 'LHR'].tail(3)
reduce_lhr_df = unique_routes_df.drop(index=lhr_to_omit.index).reset_index(drop=True)
reduce_lhr_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Region                    38 non-null     object
 1   Origin Airport Code       38 non-null     object
 2   Origin Airport Name       38 non-null     object
 3   Destination Airport Code  38 non-null     object
 4   Destination Airport Name  38 non-null     object
 5   Seats                     38 non-null     int64 
dtypes: int64(1), object(5)
memory usage: 1.9+ KB


In [9]:
count_airports(reduce_lhr_df)

Destination Airport Code  Region       
SIN                       Asia Pacific     3
LHR                       Europe           3
MAD                       Europe           2
TPE                       Asia Pacific     2
SJU                       Latin America    2
SCL                       Latin America    2
MAD                       Latin America    1
YUL                       North America    1
TPE                       North America    1
RUN                       Africa           1
Name: Seats, dtype: int64

### Step 4: Adding European routes with bad weather

In [85]:
# Stockholm -> Oslo 1089467
# Oslo -> Stockholm 1089467
# London -> Stockholm 1364019
# Stockholm -> Helsinki 1021511
# Amsterdam -> Helsinki 596139
# Berlin -> Zurich 559958

# Mallorca -> Frankfurt 418946
# Dublin -> Amsterdam 1187883
# London Heathrow -> Dublin 1856099

# Copenhagen -> Reykjavik 582199
# Reykjavik -> Paris 443312

In [10]:
new_routes = [
    {'Region': 'Europe', 'Origin Airport Code': 'PMI', 'Origin Airport Name': 'Palma de Mallorca Airport', 
     'Destination Airport Code': 'FRA', 'Destination Airport Name': 'Frankfurt Airport', 'Seats': 418946},
    {'Region': 'Europe', 'Origin Airport Code': 'DUB', 'Origin Airport Name': 'Dublin Airport', 
     'Destination Airport Code': 'AMS', 'Destination Airport Name': 'Amsterdam Schiphol Airport', 'Seats': 1187883},
    {'Region': 'Europe', 'Origin Airport Code': 'LHR', 'Origin Airport Name': 'London Heathrow Airport', 
     'Destination Airport Code': 'DUB', 'Destination Airport Name': 'Dublin Airport', 'Seats': 1856099},
    {'Region': 'Europe', 'Origin Airport Code': 'ARN', 'Origin Airport Name': 'Stockholm Arlanda Airport', 
     'Destination Airport Code': 'OSL', 'Destination Airport Name': 'Oslo Gardermoen Airport', 'Seats': 1089467},
    {'Region': 'Europe', 'Origin Airport Code': 'OSL', 'Origin Airport Name': 'Oslo Gardermoen Airport', 
     'Destination Airport Code': 'ARN', 'Destination Airport Name': 'Stockholm Arlanda Airport', 'Seats': 1089467},
    {'Region': 'Europe', 'Origin Airport Code': 'LHR', 'Origin Airport Name': 'London Heathrow Airport', 
     'Destination Airport Code': 'ARN', 'Destination Airport Name': 'Stockholm Arlanda Airport', 'Seats': 1364019},
    {'Region': 'Europe', 'Origin Airport Code': 'ARN', 'Origin Airport Name': 'Stockholm Arlanda Airport', 
     'Destination Airport Code': 'HEL', 'Destination Airport Name': 'Helsinki Airport', 'Seats': 1021511},
    {'Region': 'Europe', 'Origin Airport Code': 'AMS', 'Origin Airport Name': 'Amsterdam Schiphol Airport', 
     'Destination Airport Code': 'HEL', 'Destination Airport Name': 'Helsinki Airport', 'Seats': 596139},
    {'Region': 'Europe', 'Origin Airport Code': 'BER', 'Origin Airport Name': 'Berlin Brandenburg Airport', 
     'Destination Airport Code': 'ZRH', 'Destination Airport Name': 'Zurich Airport', 'Seats': 559958},
    {'Region': 'Europe', 'Origin Airport Code': 'CPH', 'Origin Airport Name': 'Copenhagen Airport', 
     'Destination Airport Code': 'KEF', 'Destination Airport Name': 'Reykjavik Keflavik Airport', 'Seats': 582199},
    {'Region': 'Europe', 'Origin Airport Code': 'KEF', 'Origin Airport Name': 'Reykjavik Keflavik Airport', 
     'Destination Airport Code': 'CDG', 'Destination Airport Name': 'Paris Charles de Gaulle Airport', 'Seats': 443312}
]

# Create a DataFrame from the new routes
new_routes_df = pd.DataFrame(new_routes)

# Add the new rows to reduce_lhr_df
new_european_routes_df = pd.concat([reduce_lhr_df, new_routes_df], ignore_index=True)

new_european_routes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Region                    49 non-null     object
 1   Origin Airport Code       49 non-null     object
 2   Origin Airport Name       49 non-null     object
 3   Destination Airport Code  49 non-null     object
 4   Destination Airport Name  49 non-null     object
 5   Seats                     49 non-null     int64 
dtypes: int64(1), object(5)
memory usage: 2.4+ KB


In [11]:
new_european_routes_df

,Region,Origin Airport Code,Origin Airport Name,Destination Airport Code,Destination Airport Name,Seats
0,Africa,CAI,Cairo International Airport,JED,King Abdulaziz International Airport,5469274
1,Africa,CAI,Cairo International Airport,MED,Prince Mohammad bin Abdulaziz International Ai...,1844795
2,Africa,ALG,Houari Boumediene Airport,CDG,Charles de Gaulle Airport,1393359
3,Africa,ORY,Orly Airport,RUN,Roland Garros Airport,1085706
4,Africa,CAI,Cairo International Airport,DOH,Hamad International Airport,1044048
5,Africa,ALG,Houari Boumediene Airport,ORY,Orly Airport,1021577
6,Asia Pacific,HKG,Hong Kong International Airport,TPE,Taoyuan International Airport,6781577
7,Asia Pacific,ICN,Incheon International Airport,NRT,Narita International Airport,5410456
8,Asia Pacific,KUL,Kuala Lumpur International Airport,SIN,Singapore Changi Airport,5382163
9,Asia Pacific,ICN,Incheon International Airport,KIX,Kansai International Airport,4982769


### Step 5: North America (Tropical storms)

In [88]:
# ATL Hartsfield-Jackson Atlanta International Airport -> MIA Miami International Airport
# MSY Louis Armstrong New Orleans International Airport -> ATL Hartsfield-Jackson Atlanta International Airport
# AMS -> ATL

In [12]:
top_busiest_flights = [
    {'Region': 'North America', 'Origin Airport Code': 'ATL', 'Origin Airport Name': 'Hartsfield-Jackson Atlanta International Airport',
     'Destination Airport Code': 'MIA', 'Destination Airport Name': 'Miami International Airport', 'Seats': 1033000},
    {'Region': 'North America', 'Origin Airport Code': 'MSY', 'Origin Airport Name': 'Louis Armstrong New Orleans International Airport',
     'Destination Airport Code': 'ATL', 'Destination Airport Name': 'Hartsfield-Jackson Atlanta International Airport', 'Seats': 245000},
    {'Region': 'Europe', 'Origin Airport Code': 'AMS', 'Origin Airport Name': 'Amsterdam Schiphol Airport',
    'Destination Airport Code': 'ATL','Destination Airport Name': 'Hartsfield-Jackson Atlanta International Airport', 'Seats': 794053}
]

# Convert the new routes into a DataFrame
top_busiest_df = pd.DataFrame(top_busiest_flights)

# Add the new routes to your existing DataFrame
new_na_routes_df = pd.concat([new_european_routes_df, top_busiest_df], ignore_index=True)

new_na_routes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Region                    52 non-null     object
 1   Origin Airport Code       52 non-null     object
 2   Origin Airport Name       52 non-null     object
 3   Destination Airport Code  52 non-null     object
 4   Destination Airport Name  52 non-null     object
 5   Seats                     52 non-null     int64 
dtypes: int64(1), object(5)
memory usage: 2.6+ KB


In [13]:
new_na_routes_df

,Region,Origin Airport Code,Origin Airport Name,Destination Airport Code,Destination Airport Name,Seats
0,Africa,CAI,Cairo International Airport,JED,King Abdulaziz International Airport,5469274
1,Africa,CAI,Cairo International Airport,MED,Prince Mohammad bin Abdulaziz International Ai...,1844795
2,Africa,ALG,Houari Boumediene Airport,CDG,Charles de Gaulle Airport,1393359
3,Africa,ORY,Orly Airport,RUN,Roland Garros Airport,1085706
4,Africa,CAI,Cairo International Airport,DOH,Hamad International Airport,1044048
5,Africa,ALG,Houari Boumediene Airport,ORY,Orly Airport,1021577
6,Asia Pacific,HKG,Hong Kong International Airport,TPE,Taoyuan International Airport,6781577
7,Asia Pacific,ICN,Incheon International Airport,NRT,Narita International Airport,5410456
8,Asia Pacific,KUL,Kuala Lumpur International Airport,SIN,Singapore Changi Airport,5382163
9,Asia Pacific,ICN,Incheon International Airport,KIX,Kansai International Airport,4982769


### Step 6: Fill up to 60 with Return Routes

In [14]:
# Identify return flights
return_flights = pd.merge(
    new_na_routes_df,  # Assuming your DataFrame is named 'df'
    new_na_routes_df,
    left_on=['Origin Airport Code', 'Destination Airport Code'],
    right_on=['Destination Airport Code', 'Origin Airport Code'],
    suffixes=('_original', '_return')
)

# Display return flights
return_flights[['Origin Airport Code_original', 'Destination Airport Code_original',
                      'Origin Airport Code_return', 'Destination Airport Code_return']]

,Origin Airport Code_original,Destination Airport Code_original,Origin Airport Code_return,Destination Airport Code_return
0,DUB,LHR,LHR,DUB
1,LHR,DUB,DUB,LHR
2,ARN,OSL,OSL,ARN
3,OSL,ARN,ARN,OSL


In [15]:
new_na_routes_df.sort_values(by='Seats',ascending=False)

,Region,Origin Airport Code,Origin Airport Name,Destination Airport Code,Destination Airport Name,Seats
6,Asia Pacific,HKG,Hong Kong International Airport,TPE,Taoyuan International Airport,6781577
0,Africa,CAI,Cairo International Airport,JED,King Abdulaziz International Airport,5469274
7,Asia Pacific,ICN,Incheon International Airport,NRT,Narita International Airport,5410456
8,Asia Pacific,KUL,Kuala Lumpur International Airport,SIN,Singapore Changi Airport,5382163
9,Asia Pacific,ICN,Incheon International Airport,KIX,Kansai International Airport,4982769
10,Asia Pacific,BKK,Suvarnabhumi Airport,HKG,Hong Kong International Airport,4201802
11,Asia Pacific,CGK,Soekarno–Hatta International Airport,SIN,Singapore Changi Airport,4069071
12,Asia Pacific,BKK,Suvarnabhumi Airport,SIN,Singapore Changi Airport,4033344
16,Europe,JFK,John F. Kennedy International Airport,LHR,Heathrow Airport,4011235
13,Asia Pacific,NRT,Narita International Airport,TPE,Taoyuan International Airport,3970518


In [16]:
new_na_routes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Region                    52 non-null     object
 1   Origin Airport Code       52 non-null     object
 2   Origin Airport Name       52 non-null     object
 3   Destination Airport Code  52 non-null     object
 4   Destination Airport Name  52 non-null     object
 5   Seats                     52 non-null     int64 
dtypes: int64(1), object(5)
memory usage: 2.6+ KB


In [ ]:
# Two most seats in Asia
# Asia Pacific	HKG	Hong Kong International Airport	TPE	Taoyuan International Airport	6781577
# Asia Pacific	ICN	Incheon International Airport	NRT	Narita International Airport	5410456

# Latin America
# Latin America	MCO	Orlando International Airport	SJU	Luis Muñoz Marín International Airport	2309653
# Latin America	LIM	Jorge Chávez International Airport	SCL	Comodoro Arturo Merino Benítez International A...	2056486

# Middle East
# Middle East	DXB	Dubai International Airport	JED	King Abdulaziz International Airport	2957019

# North America
# North America	SFO	San Francisco International Airport	TPE	Taoyuan International Airport	1709290
# North America	CDG	Charles de Gaulle Airport	YUL	Montréal–Trudeau International Airport	1515224

# Africa
# Africa	CAI	Cairo International Airport	JED	King Abdulaziz International Airport	5469274

In [18]:
# Define the return flights
return_flights = [
    # Asia
    {'Region': 'Asia Pacific', 'Origin Airport Code': 'TPE', 'Origin Airport Name': 'Taoyuan International Airport', 
     'Destination Airport Code': 'HKG', 'Destination Airport Name': 'Hong Kong International Airport', 'Seats': 6781577},
    {'Region': 'Asia Pacific', 'Origin Airport Code': 'NRT', 'Origin Airport Name': 'Narita International Airport', 
     'Destination Airport Code': 'ICN', 'Destination Airport Name': 'Incheon International Airport', 'Seats': 5410456},
    # Latin America
    {'Region': 'Latin America', 'Origin Airport Code': 'SJU', 'Origin Airport Name': 'Luis Muñoz Marín International Airport', 
     'Destination Airport Code': 'MCO', 'Destination Airport Name': 'Orlando International Airport', 'Seats': 2309653},
    {'Region': 'Latin America', 'Origin Airport Code': 'SCL', 'Origin Airport Name': 'Comodoro Arturo Merino Benítez International Airport', 
     'Destination Airport Code': 'LIM', 'Destination Airport Name': 'Jorge Chávez International Airport', 'Seats': 2056486},
    # Middle East
    {'Region': 'Middle East', 'Origin Airport Code': 'JED', 'Origin Airport Name': 'King Abdulaziz International Airport', 
     'Destination Airport Code': 'DXB', 'Destination Airport Name': 'Dubai International Airport', 'Seats': 2957019},
    # North America
    {'Region': 'North America', 'Origin Airport Code': 'TPE', 'Origin Airport Name': 'Taoyuan International Airport', 
     'Destination Airport Code': 'SFO', 'Destination Airport Name': 'San Francisco International Airport', 'Seats': 1709290},
    {'Region': 'North America', 'Origin Airport Code': 'YUL', 'Origin Airport Name': 'Montréal–Trudeau International Airport', 
     'Destination Airport Code': 'CDG', 'Destination Airport Name': 'Charles de Gaulle Airport', 'Seats': 1515224},
    # Africa
    {'Region': 'Africa', 'Origin Airport Code': 'JED', 'Origin Airport Name': 'King Abdulaziz International Airport', 
     'Destination Airport Code': 'CAI', 'Destination Airport Name': 'Cairo International Airport', 'Seats': 5469274},
]

# Convert the return flights into a DataFrame
return_flights_df = pd.DataFrame(return_flights)

# Append the return flights to the existing DataFrame
final_df = pd.concat([new_na_routes_df, return_flights_df], ignore_index=True)

# Display the updated DataFrame
final_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Region                    60 non-null     object
 1   Origin Airport Code       60 non-null     object
 2   Origin Airport Name       60 non-null     object
 3   Destination Airport Code  60 non-null     object
 4   Destination Airport Name  60 non-null     object
 5   Seats                     60 non-null     int64 
dtypes: int64(1), object(5)
memory usage: 2.9+ KB


### Step 7: Add routes suggested by stakeholder

In [24]:
## Add Jakarta (code CKG) to complete the dataset featuring Asia 
## Add more flights inside North America. The dataset currently overrepresents international flights to North America. i.e. from/to the US/Canada east coast in particular, which is seasonal bad weather.
new_routes_v3 = [
    {'Region': 'North America', 'Origin Airport Code': 'LGA', 'Origin Airport Name': 'New York LaGuardia Airport', 
     'Destination Airport Code': 'ORD', 'Destination Airport Name': "Chicago O'Hare International Airport", 'Seats': 3118179},
    {'Region': 'North America', 'Origin Airport Code': 'YVR', 'Origin Airport Name': 'Vancouver International Airport', 
     'Destination Airport Code': 'YYZ', 'Destination Airport Name': 'Toronto Pearson International Airport', 'Seats': 3498835},
    {'Region': 'North America', 'Origin Airport Code': 'JFK', 'Origin Airport Name': 'New York John F. Kennedy International Airport', 
     'Destination Airport Code': 'MCO', 'Destination Airport Name': 'Orlando International Airport', 'Seats': 3494400},
    {'Region': 'Asia Pacific', 'Origin Airport Code': 'DEL', 'Origin Airport Name': 'Delhi Indira Gandhi International Airport', 
     'Destination Airport Code': 'BOM', 'Destination Airport Name': 'Mumbai Chhatrapati Shivaji International Airport', 'Seats': 7963686},
    {'Region': 'Asia Pacific', 'Origin Airport Code': 'CGK', 'Origin Airport Name': 'Jakarta Soekarno-Hatta International Airport', 
     'Destination Airport Code': 'SIN', 'Destination Airport Name': 'Singapore Changi Airport', 'Seats': 4070000}
]

# Create a DataFrame from the new routes
new_routes_v3_df = pd.DataFrame(new_routes_v3)
final_df_v3 = pd.concat([final_df, new_routes_v3_df], ignore_index=True)
final_df_v3.info()




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Region                    65 non-null     object
 1   Origin Airport Code       65 non-null     object
 2   Origin Airport Name       65 non-null     object
 3   Destination Airport Code  65 non-null     object
 4   Destination Airport Name  65 non-null     object
 5   Seats                     65 non-null     int64 
dtypes: int64(1), object(5)
memory usage: 3.2+ KB


### Step 8: Export to csv

In [25]:
final_df_v3.to_csv('../data/raw/routes_by_region_2024_v3.csv')